In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import math
import os
import matplotlib.image as img
import scipy.io
import pickle
from sklearn.metrics import pairwise_distances
import time
from sklearn import linear_model
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix

### Step 0: set work directories

In [2]:
np.random.seed(2020)

Provide directories for training images. Training images and Training fiducial points will be in different subfolders.

In [3]:
#Change train_dir to where you have the train set because we can't
#upload the train set to github
train_dir = "/Users/rohan/Desktop/train_set/"
train_image_dir = train_dir+"images/"
train_pt_dir = train_dir+"points/"
train_label_path = train_dir+"label.csv"

### Step 1: set up controls for evaluation experiments

In this chunk, we have a set of controls for the evaluation experiments. 

+ (T/F) cross-validation on the training set
+ (T/F) reweighting the samples for training set 
+ (number) K, the number of CV folds
+ (T/F) process features for training set
+ (T/F) run evaluation on an independent test set
+ (T/F) process features for test set

In [4]:
run_cv = True # run cross-validation on the training set
sample_reweight = True # run sample reweighting in model training
K = 5  # number of CV folds
run_feature_train = True # process features for training set
run_test = True # run evaluation on an independent test set
run_feature_test = True # process features for test set

Using cross-validation or independent test set evaluation, we compare the performance of models with different specifications. In this Starter Code, we tune parameter lambda (the amount of shrinkage) for logistic regression with LASSO penalty.

In [5]:
#Instead of using these lambda for LASSO like in the starter code
#we'll use these lambda for improving the baseline gradient boosting model

lmbd = [1e-3, 5e-3, 1e-2, 5e-2, 1e-1]
model_labels = ["Gradient Boosting with learning rate = "+str(x) for x in lmbd]

### Step 2: import data and train-test split

In [6]:
info = pd.read_csv(train_label_path)
n = info.shape[0]

#we could use sklearn train_test_split here instead of doing it
#manually like in the starter code
n_train = int(round(n*(4/5),0))
train_idx = np.random.choice(list(info.index),size=n_train,replace=False)
test_idx = list(set(list(info.index))-set(train_idx)) #set difference

If you choose to extract features from images, such as using Gabor filter, R memory will exhaust all images are read together. The solution is to repeat reading a smaller batch(e.g 100) and process them. 

In [7]:
n_files = len(os.listdir(train_image_dir))

image_list = []
for i in range(1,101): # 1 to 100
    image = img.imread(train_image_dir+'{:04d}'.format(i)+'.jpg')
    image_list.append(image)

Fiducial points are stored in matlab format. In this step, we read them and store them in a list.

In [8]:
#function to read fiducial points
#input: index
#output: matrix of fiducial points corresponding to the index

def readMat_matrix(index):
    try:
        mat_data = scipy.io.loadmat(train_pt_dir+'{:04d}'.format(index)+'.mat')['faceCoordinatesUnwarped']
    except KeyError:
        mat_data = scipy.io.loadmat(train_pt_dir+'{:04d}'.format(index)+'.mat')['faceCoordinates2']
    return np.matrix.round(mat_data,0)

#load fiducial points
#pickle is the closest equivalent to .RData that I could find in Python
fiducial_pt_list = list(map(readMat_matrix,list(range(1,n_files+1))))
pickle.dump(fiducial_pt_list, open( "../output/fiducial_pt_list.p", "wb" ) )

### Step 3: construct features and responses

+ The follow plots show how pairwise distance between fiducial points can work as feature for facial emotion recognition.

  + In the first column, 78 fiducials points of each emotion are marked in order. 
  + In the second column distributions of vertical distance between right pupil(1) and  right brow peak(21) are shown in  histograms. For example, the distance of an angry face tends to be shorter than that of a surprised face.
  + The third column is the distributions of vertical distances between right mouth corner(50)
and the midpoint of the upper lip(52).  For example, the distance of an happy face tends to be shorter than that of a sad face.

![Figure1](../figs/feature_visualization.jpg)

`feature.R` should be the wrapper for all your feature engineering functions and options. The function `feature( )` should have options that correspond to different scenarios for your project and produces an R object that contains features and responses that are required by all the models you are going to evaluate later. 
  
  + `feature.R`
  + Input: list of images or fiducial point
  + Output: an RData file that contains extracted features and corresponding responses

In [9]:
#TO DO: make this function into its own file feature.ipynb 
#since feature.R is a separate file in the starter code

def feature(index, input_list = fiducial_pt_list):
    ### Construct process features for training images 

    ### Input: a list of images or fiducial points; index: train index or test index

    ### Output: a data frame containing: features and a column of label

    ### here is an example of extracting pairwise distances between fiducial points
    ### Step 1: Write a function pairwise_dist to calculate pairwise distance of items in a vector
    def pairwise_dist(vec):
        n = len(vec)
        dist_matrix = pairwise_distances(np.array(vec).reshape(-1,1))
        return list(dist_matrix[np.triu_indices(n,k=1)])
    
    ### Step 2: Write a function pairwise_dist_result to apply function in Step 1 to column of a matrix 
    def pairwise_dist_result(mat):
        ### input: a n*2 matrix(e.g. fiducial_pt_list[[1]]), output: a vector(length n(n-1))
        return list(np.transpose(np.apply_along_axis(pairwise_dist,0,mat)).flatten())
    
    ### Step 3: Apply function in Step 2 to selected index of input list, output: a feature matrix with ncol = n(n-1) = 78*77 = 6006
    pairwise_dist_feature = ((np.array(list(map(pairwise_dist_result, [input_list[i] for i in index])))))
    pairwise_dist_feature.shape
    
    colnames = ['feature'+str(i) for i in range(pairwise_dist_feature.shape[1])]
    df = pd.DataFrame(pairwise_dist_feature,columns=colnames)
    label_df = pd.DataFrame(list(info['label'].iloc[index]),columns=['labels'])
    
    pairwise_data = pd.concat([df,label_df],axis=1)
    return pairwise_data

In [10]:
tm_feature_train = np.nan
if run_feature_train == True:
    start = time.time()
    dat_train = feature(train_idx, fiducial_pt_list)
    end = time.time()
    tm_feature_train = end-start
    pickle.dump(dat_train, open( "../output/feature_train.p", "wb" ) )
else:
    pickle.load(open("../output/feature_train.p", "rb"))
    
    
tm_feature_test = np.nan
if run_feature_test == True:
    start = time.time()
    dat_test = feature(test_idx, fiducial_pt_list)
    end = time.time()
    tm_feature_test = end-start
    pickle.dump(dat_test, open( "../output/feature_test.p", "wb" ) )
else:
    pickle.load(open("../output/feature_test.p", "rb"))
    

### Step 4: Train a classification model with training features and responses

Call the train model and test model from library. 

`train.R` and `test.R` should be wrappers for all your model training steps and your classification/prediction steps. 

+ `train.R`
  + Input: a data frame containing features and labels and a parameter list.
  + Output:a trained model
+ `test.R`
  + Input: the fitted classification model using training data and processed features from testing images 
  + Input: an R object that contains a trained classifier.
  + Output: training model specification

+ In this Starter Code, we use logistic regression with LASSO penalty to do classification.

#### Model selection with cross-validation
* Do model selection by choosing among different values of training model parameters.

In [11]:

"""
Problem : sklearn lasso function doesn't have a weights argument.
In the starter code they use R's glmnet which does have weights argument.
We need to implement gbm instead of lasso for baseline model anyway
so this isn't really a big issue.


def train(features, labels, w = np.nan, l = 1):
    model = linear_model.Lasso(alpha = l)
    model.fit(features,labels)
"""



"""
#implementing GBM instead of lasso
def train(features, labels, l = 1):
    model = GradientBoostingClassifier(n_estimators=50,max_depth= 10,learning_rate=l)
    model = model.fit(train_x,train_y) 
    model.fit(features,labels)
    return model
"""
    

'\n#implementing GBM instead of lasso\ndef train(features, labels, l = 1):\n    model = GradientBoostingClassifier(n_estimators=50,max_depth= 10,learning_rate=l)\n    model = model.fit(train_x,train_y) \n    model.fit(features,labels)\n    return model\n'

In [12]:
"""
Maybe we can just use grid search and model scores from sklearn
instead of defining our own cv_function

Starter code cv_function (UNFINISHED)

def cv_function(features, labels, K, l, reweight = False):
    ### Input:
    ### - features: feature data frame
    ### - labels: label data vector
    ### - K: a number stands for K-fold CV
    ### - l: tuning parameters 
    ### - reweight: sample reweighting 
    
    random.seed(2020)
    n = features.shape[0]
    n_fold = int(round(n/K,0))
    s = 1+(np.random.choice(list(range(1,n+1)),size=n,replace=False)%K)
    cv_error = [np.nan]*K
    cv_AUC = [np.nan]*K
    
    for i in range(0,K):
        ## create features and labels for train/test
        feature_train = features.iloc[s != i]
        feature_test = features.iloc[s == i]
        label_train = [x for x in labels if s != i]
        label_test = [x for x in labels if s == i]
    
    weight_train = [np.nan]*label_train.shape[0]
    weight_test = [np.nan]*label_test.shape[0]
    
    fow
"""
    

'\nMaybe we can just use grid search and model scores from sklearn\ninstead of defining our own cv_function\n\nStarter code cv_function (UNFINISHED)\n\ndef cv_function(features, labels, K, l, reweight = False):\n    ### Input:\n    ### - features: feature data frame\n    ### - labels: label data vector\n    ### - K: a number stands for K-fold CV\n    ### - l: tuning parameters \n    ### - reweight: sample reweighting \n    \n    random.seed(2020)\n    n = features.shape[0]\n    n_fold = int(round(n/K,0))\n    s = 1+(np.random.choice(list(range(1,n+1)),size=n,replace=False)%K)\n    cv_error = [np.nan]*K\n    cv_AUC = [np.nan]*K\n    \n    for i in range(0,K):\n        ## create features and labels for train/test\n        feature_train = features.iloc[s != i]\n        feature_test = features.iloc[s == i]\n        label_train = [x for x in labels if s != i]\n        label_test = [x for x in labels if s == i]\n    \n    weight_train = [np.nan]*label_train.shape[0]\n    weight_test = [np.na

### Baseline Model

In [13]:
feature_train = dat_train.loc[:, dat_train.columns != 'labels']
label_train = dat_train['labels']

feature_test = dat_test.loc[:, dat_test.columns != 'labels']
label_test = dat_test['labels']

In [14]:
#need to do a grid search for optimal parameters
#takes a really long time and we don't know the features yet
#so i commented the grid search out for now
if (run_cv):
    params = {'learning_rate':lmbd, 'max_depth': [1,2,3,4], 'n_estimators':[100,200,300,400,500]}
    #gscv = GridSearchCV(GradientBoostingClassifier(),params,cv=K).fit(feature_train,label_train)


In [15]:
#Baseline model
#need to cross validate to get optimal parameters though

start = time.time()
gbm=GradientBoostingClassifier(learning_rate=0.1,max_depth=2,n_estimators=100)
gbm.fit(feature_train,label_train)
end = time.time()

print('Training time {:4f} seconds'.format(end-start))

Training time 82.357947 seconds


In [16]:
test_preds=gbm.predict(feature_test)
np.mean(np.array(test_preds)!=np.array(label_test)) #Classification Error

0.18833333333333332

In [17]:
gbm.score(feature_test,label_test)
#score is 1-classification error i.e.
#1-(np.mean(np.array(test_preds)!=np.array(label_test)))

0.8116666666666666

In [18]:
print(classification_report(label_test,test_preds))

              precision    recall  f1-score   support

           0       0.81      0.99      0.89       473
           1       0.75      0.17      0.27       127

    accuracy                           0.81       600
   macro avg       0.78      0.58      0.58       600
weighted avg       0.80      0.81      0.76       600



In [19]:
confusion_matrix(label_test,test_preds)

array([[466,   7],
       [106,  21]])